In [1]:
"""
A couple of years ago, J.K. Rowling (of Harry Potter fame) tried something interesting. She wrote a book, "The Cuckoo's Calling," 
under the name Robert Galbraith. The book received some good reviews, but no one paid much attention to it--until an anonymous 
tipster on Twitter said it was J.K. Rowling. The London Sunday Times enlisted two experts to compare the linguistic patterns of 
"Cuckoo" to Rowling's "The Casual Vacancy," as well as to books by several other authors. After the results of their analysis 
pointed strongly toward Rowling as the author, the Times directly asked the publisher if they were the same person, and the 
publisher confirmed. The book exploded in popularity overnight.

We'll do something very similar in this project. We have a set of emails, half of which were written by one person and the other 
half by another person at the same company. Our objective is to classify the emails as written by one person or the other based 
only on the text of the email. We will start with Naive Bayes in this mini-project, and then expand in later projects to other 
algorithms.

We will start by giving you a list of strings. Each string is the text of an email, which has undergone some basic preprocessing; 
we will then provide the code to split the dataset into training and testing sets. (In the next lessons you'll learn how to do this 
preprocessing and splitting yourself, but for now we'll give the code to you).

One particular feature of Naive Bayes is that it's a good algorithm for working with text classification. When dealing with text, 
it's very common to treat each unique word as a feature, and since the typical person's vocabulary is many thousands of words, 
this makes for a large number of features. The relative simplicity of the algorithm and the independent features assumption of 
Naive Bayes make it a strong performer for classifying texts. In this mini-project, you will download and install sklearn on your 
computer and use Naive Bayes to classify emails by author.
"""

#!/usr/bin/python

import pickle
import cPickle
import numpy

from sklearn import cross_validation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif


def preprocess(words_file = "word_data.pkl", authors_file="email_authors.pkl"):
    """ 
        this function takes a pre-made list of email texts (by default word_data.pkl)
        and the corresponding authors (by default email_authors.pkl) and performs
        a number of preprocessing steps:
            -- splits into training/testing sets (10% testing)
            -- vectorizes into tfidf matrix
            -- selects/keeps most helpful features

        after this, the feaures and labels are put into numpy arrays, which play nice with sklearn functions

        4 objects are returned:
            -- training/testing features
            -- training/testing labels

    """

    ### the words (features) and authors (labels), already largely preprocessed
    ### this preprocessing will be repeated in the text learning mini-project
    authors_file_handler = open(authors_file, "r")
    authors = pickle.load(authors_file_handler)
    authors_file_handler.close()

    words_file_handler = open(words_file, "r")
    word_data = cPickle.load(words_file_handler)
    words_file_handler.close()

    ### test_size is the percentage of events assigned to the test set
    ### (remainder go into training)
    features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)



    ### text vectorization--go from strings to lists of numbers
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
    features_train_transformed = vectorizer.fit_transform(features_train)
    features_test_transformed  = vectorizer.transform(features_test)



    ### feature selection, because text is super high dimensional and 
    ### can be really computationally chewy as a result
    selector = SelectPercentile(f_classif, percentile=10)
    selector.fit(features_train_transformed, labels_train)
    features_train_transformed = selector.transform(features_train_transformed).toarray()
    features_test_transformed  = selector.transform(features_test_transformed).toarray()

    ### info on the data
    print "no. of Chris training emails:", sum(labels_train)
    print "no. of Sara training emails:", len(labels_train)-sum(labels_train)
    
    return features_train_transformed, features_test_transformed, labels_train, labels_test

In [2]:
#!/usr/bin/python

""" 
    This is the code to accompany the Lesson 1 (Naive Bayes) mini-project. 

    Use a Naive Bayes Classifier to identify emails by their authors
    
    authors and labels:
    Sara has label 0
    Chris has label 1
"""
    
import sys
from time import time
#sys.path.append("../tools/")
#from email_preprocess import preprocess


### features_train and features_test are the features for the training
### and testing datasets, respectively
### labels_train and labels_test are the corresponding item labels
features_train, features_test, labels_train, labels_test = preprocess()


no. of Chris training emails: 7936
no. of Sara training emails: 7884


In [4]:
"""
Create and train a Naive Bayes classifier in naive_bayes/nb_author_id.py. Use it to make predictions for the test set. 
What is the accuracy?

When training you may see the following error: UserWarning: Duplicate scores. Result may depend on feature ordering. There are 
probably duplicate features, or you used a classification score for a regression task. warn("Duplicate scores. Result may depend 
on feature ordering.")

This is a warning that two or more words happen to have the same usage patterns in the emails--as far as the algorithm is 
concerned, this means that two features are the same. Some algorithms will actually break (mathematically won't work) or give 
multiple different answers (depending on feature ordering) when there are duplicate features and sklearn is giving us a warning. 
Good information, but not something we have to worry about.
"""
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

### create classifier
clf = GaussianNB()

### fit the classifier on the training features and labels
clf.fit(features_train, labels_train)

### use the trained classifier to predict labels for the test features
pred = clf.predict(features_test)

### calculate and return the accuracy on the test data
### this is slightly different than the example, 
### where we just print the accuracy
### you might need to import an sklearn module
accuracy = accuracy_score(pred, labels_test)
print 'Accuracy:', accuracy


Accuracy: 0.973265073948


In [5]:
"""
An important topic that we didn't explicitly talk about is the time to train and test our algorithms. Put in two lines of code, 
above and below the line fitting your classifier, like this:

t0 = time()
< your clf.fit() line of code >
print "training time:", round(time()-t0, 3), "s"

Put similar lines of code around the clf.predict() line of code, so you can compare the time to train the classifier and to make 
predictions with it. What is faster, training or prediction?

We will compare the Naive Bayes timing to a couple other algorithms, so note down the speed and accuracy you get and we’ll revisit 
this in the next mini-project.
"""
clf = GaussianNB()

t0 = time()
clf.fit(features_train, labels_train)
print "training time:", round(time()-t0, 3), "s"

t0 = time()
pred = clf.predict(features_test)
print "predicting time:", round(time()-t0, 3), "s"


training time: 1.254 s
predicting time: 0.189 s
